# GhanaSegNet Colab Training Notebook
This notebook sets up your environment, installs dependencies, and runs your baseline training script for UNet, DeepLabV3+, and SegFormer-B0 on Colab GPU.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone your GitHub repo (or copy from Drive)
!git clone https://github.com/EricBaidoo/GhanaSegNet.git
%cd GhanaSegNet

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Run baseline training for all models
!python scripts/train_baselines.py --model all --epochs 1

In [ ]:
# Optionally, copy checkpoints/results to Google Drive
!cp -r checkpoints /content/drive/MyDrive/GhanaSegNet_checkpoints